In [2]:
from pycocotools.coco import COCO
import pandas as pd

In [3]:
df = pd.DataFrame(columns=['file_name', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax'])
df = df.astype({"file_name": "string",'width': 'int16', 'height': 'int16', 'class': 'int8'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  0 non-null      string
 1   width      0 non-null      int16 
 2   height     0 non-null      int16 
 3   class      0 non-null      int8  
 4   xmin       0 non-null      object
 5   ymin       0 non-null      object
 6   xmax       0 non-null      object
 7   ymax       0 non-null      object
dtypes: int16(2), int8(1), object(4), string(1)
memory usage: 124.0+ bytes


In [7]:
df = pd.read_csv(r'C:\Users\caretaker\Documents\way-to-ml\hack2023urban\data.csv')
df

,file_name,width,height,class,xmin,ymin,xmax,ymax
0,0000019949building.jpg,854,572,2,98.87,141.25,209.50,255.84
1,0000019949building.jpg,854,572,2,723.17,184.07,854.00,308.50
2,0000019949building.jpg,854,572,2,475.70,48.84,592.32,168.41
3,0000019949building.jpg,854,572,2,593.81,62.38,712.70,178.85
4,0000019949building.jpg,854,572,2,0.00,259.95,81.34,393.50
...,...,...,...,...,...,...,...,...
38326,20221212_144514.jpg,4160,3120,2,0.00,1383.28,480.68,1968.78
38327,20221212_144514.jpg,4160,3120,2,0.00,919.61,630.00,1387.21
38328,20221212_144514.jpg,4160,3120,2,119.17,510.94,743.96,927.47
38329,20221212_144514.jpg,4160,3120,2,268.49,173.01,842.20,507.01


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38331 entries, 0 to 38330
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   file_name  38331 non-null  object 
 1   width      38331 non-null  int64  
 2   height     38331 non-null  int64  
 3   class      38331 non-null  int64  
 4   xmin       38331 non-null  float64
 5   ymin       38331 non-null  float64
 6   xmax       38331 non-null  float64
 7   ymax       38331 non-null  float64
dtypes: float64(4), int64(3), object(1)
memory usage: 2.3+ MB


In [4]:
coco = COCO(annotation_file='../hack2023urban/urbanhack-train/annotations/instances_default.json')

catIds = coco.getCatIds()
imgIds = coco.getImgIds()

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: '../hack2023urban/urbanhack-train/annotations/instances_default.json'

In [ ]:
image_example = coco.loadImgs(imgIds[0])
image_example

In [ ]:
annot_example = coco.loadAnns(coco.getAnnIds(imgIds=1))
annot_example

In [1]:
for img_index in imgIds:
    img = coco.loadImgs(imgIds[img_index-1])[0]
    new_df_row = []
    new_df_row.extend([img['file_name'], img['width'], img['height']])
    for annot in coco.loadAnns(coco.getAnnIds(imgIds=img_index)):  
        tmp = new_df_row.copy()
        tmp.extend([annot['category_id'], annot['bbox'][0], annot['bbox'][1], annot['bbox'][0]+annot['bbox'][2], annot['bbox'][1]+annot['bbox'][3]])
        df.loc[len(df.index)] = tmp
    
    

df

NameError: name 'imgIds' is not defined

In [ ]:
import os
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET
import argparse

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import tensorflow.compat.v1 as tf
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple


label_map = label_map_util.load_labelmap(к'C:\Users\caretaker\Documents\way-to-ml\hack2023urban\model\label_map.pbtxt')
label_map_dict = label_map_util.get_label_map_dict(label_map)

def class_text_to_int(row_label):
    return label_map_dict[row_label]


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        # classes_text.append(.encode('utf8'))
        classes.append(row['class'])

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

writer = tf.python_io.TFRecordWriter('/content/DATA_VAL.tfrecord')
path = os.path.join('/content/drive/MyDrive/Tensorflow/hack/')
examples_all = df
#change .head to .tail for train and val datasets
examples = examples_all.tail(int(len(examples_all)*(20/100)))
grouped = split(examples, 'file_name')
for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())
writer.close()
print('Successfully created the TFRecord file')



